In [1]:
#Uncomment if you need to install these packages
#pip install pslpython
#pip install ipywidgets

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug  4 09:49:29 2022

@author: ewanhilton
"""

from classes.DatasetGenerator import generate_dataset_file
from classes.EntityConverter import EntityConverter
from classes.PSLFileBuilder import PSLFileBuilder
from classes.Dataset import get_dataset
from classes.Dataset import get_dataset_name

#Setting this to True is required to get all files needed for PSL,
#but is very costly
CREATE_FILES = True 

def pre_main():
    dataset = get_dataset()
    
    train_triples = dataset.training.mapped_triples.numpy()
    val_triples = dataset.validation.mapped_triples.numpy()
    test_triples = dataset.testing.mapped_triples.numpy()   
    dataset_name = get_dataset_name()
    generate_dataset_file('train.txt',dataset_name,train_triples,dataset)
    generate_dataset_file('valid.txt',dataset_name,val_triples,dataset)
    generate_dataset_file('test.txt',dataset_name,test_triples,dataset)
    
    if CREATE_FILES:       
        train_triples = dataset.training.mapped_triples.numpy()
        val_triples = dataset.validation.mapped_triples.numpy()
        #test_triples = dataset.testing.mapped_triples.numpy()  
    
        entity_converter = EntityConverter(dataset)
        create_files(train_triples,val_triples,entity_converter)
    
    #Create files needed by PSL
def create_files(train_triples, val_triples,entity_converter):    
    filebuilder = PSLFileBuilder(train_triples, val_triples, entity_converter)
    filebuilder.build_map_files()
    filebuilder.build_obs_files()
    #filebuilder.build_target_files()
    filebuilder.build_truth_files()

/home/ewan/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug  4 08:04:34 2022

@author: ewanhilton
"""
import os

from pslpython.model import Model
from pslpython.partition import Partition
from pslpython.predicate import Predicate
from pslpython.rule import Rule
from classes.Dataset import get_dataset
from classes.ANYBurlToPSLConverter import ANYBurlToPSLConverter
from classes.RuleImporter import RuleImporter
from classes.EntityConverter import EntityConverter
from classes.DatasetGenerator import encode_text
from tqdm import tqdm
from datetime import datetime
from classes.PSLFileBuilder import build_target_files
from classes.DatasetGenerator import encode_text
from classes.PSLFileBuilder import PSLFileBuilder
import glob

MODEL_NAME = 'ANYBurl and PSL Model'

DATA_DIR = os.path.join('data')

ADDITIONAL_PSL_OPTIONS = {
    'log4j.threshold': 'INFO'
}

ADDITIONAL_CLI_OPTIONS = [
    # '--postgres'
]

MAX_RULES = None
ANYBURL_RULES_THRESHOLD = 0.6
PREDICT_PREDICATES_ONE_AT_A_TIME = False

def main():
    print(f"PSL run stated at {get_date_time()}")
    importer = RuleImporter()
    importer.import_rules()
   
    dataset = get_dataset()
    entity_converter = EntityConverter(dataset)

    train_triples = dataset.training.mapped_triples.numpy()
    val_triples = dataset.validation.mapped_triples.numpy()
    #test_triples = dataset.testing.mapped_triples.numpy()  
    
    #Delete all pre-existing predicates from the folder
    for f in glob.glob("inferred-predicates/*"):
                    os.remove(f)
            
    if PREDICT_PREDICATES_ONE_AT_A_TIME :
        for relindex,name in tqdm(entity_converter.relindex_to_name.items()): 
            try:
                model = Model(MODEL_NAME)
                
                #Delete existing targets files so we only deal with one at a time
                for f in glob.glob("data/targets/*"):
                    os.remove(f)
                
                #Add Target Files
                build_target_files(name,relindex, entity_converter, train_triples)
                # Add Predicates
                add_predicates(model,entity_converter,name)

                # Add Rules
                add_rules(model,importer.rules, encode_text(name), name)

                # Add Data
                add_data(model, entity_converter, name)

                # Inference
                results = infer(model)
                write_results(results, model)
            except Exception as e: 
                print(e)
                continue
    else:
        name = "All Predicates"
        model = Model(MODEL_NAME)

        #Add Target Files
        filebuilder = PSLFileBuilder(train_triples, val_triples, entity_converter)
        filebuilder.build_target_files()
        # Add Predicates
        add_predicates(model,entity_converter)

        # Add Rules
        add_rules(model,importer.rules, None)

        # Add Data
        add_data(model, entity_converter)

        # Inference
        results = infer(model)
        write_results(results, model)
    print(f"PSL run ended at {get_date_time()}")
    
def add_predicates(model,entity_converter,target_name=None):
    if target_name is not None:
        print(f"Adding predicates for {target_name}")
    else:
        print("Adding predicates for All Predicates")
    for relindex,name in tqdm(entity_converter.relindex_to_name.items()):
        is_closed = False
        if target_name is not None:
            is_closed = name != target_name
        predicate = Predicate(encode_text(name), closed = is_closed, size = 2)
        model.add_predicate(predicate)

def add_rules(model, rules, name, target_name=None):
    if target_name is not None:
        print(f"Adding rules for {target_name}")
    else:
        print("Adding rules for All Predicates")
    converter = ANYBurlToPSLConverter(rules)
    total_rules = 0
    for rule in tqdm(converter.converted_rules):
        if name is None or rule.split('->')[1].split('(')[0].replace(' ','') == name:
            if MAX_RULES != None and total_rules >= MAX_RULES:
                print(f"Maximum number of rules added ({total_rules} rules added)")
                return
            if float(rule.split(':')[0]) > ANYBURL_RULES_THRESHOLD:
                model.add_rule(Rule(rule))
                total_rules += 1
                continue
                
    if total_rules < 1:
        raise Exception(f"Failed to add any rules. Aborting inference for {target_name}")
    print(f"{total_rules} rules added")
    
def add_data(model,entity_converter, target_name = None):
    if target_name is not None:
        print(f"Adding data for {target_name}")
    else:
        print("Adding data for All Predicates")
        
    for relindex,name in tqdm(entity_converter.relindex_to_name.items()):        
        path = f'data/obs/{encode_text(name)}_obs.txt'
        if path_exists(path): #Check file has content before adding  
            model.get_predicate(encode_text(name)).add_data_file(Partition.OBSERVATIONS, path)

        path =  f'data/targets/{encode_text(name)}_targets.txt'
        if path_exists(path):
            model.get_predicate(encode_text(name)).add_data_file(Partition.TARGETS, path)

        path = f'data/truth/{encode_text(name)}_truth.txt'
        if path_exists(path):
            model.get_predicate(encode_text(name)).add_data_file(Partition.TRUTH, path)
   
def path_exists(path):
    try:
        return os.path.getsize(f"{path}") > 0
    except:
        return False

def infer(model): 
    print(f"Inference starting at {get_date_time()}")
    return model.infer(psl_config = ADDITIONAL_PSL_OPTIONS)
     
def get_date_time():
    return f"{str(datetime.now().time()).split('.')[0]} on {datetime.today().strftime('%d-%b-%Y')}"

def write_results(results, model):
    print(f"Inferenced completed at {get_date_time()}")
    out_dir = 'inferred-predicates'
    os.makedirs(out_dir, exist_ok = True)
    print("Writing predicates")
    for predicate in tqdm(model.get_predicates().values()):
        if (predicate.closed()):
            continue       
        try:
            out_path = os.path.join(out_dir, "%s.txt" % (predicate.name()))
            results[predicate].to_csv(out_path, sep = "\t", header = False, index = False)     
        except:
            continue

In [4]:
pre_main()

Generating Nations train.txt dataset file in destination: datasets/data/Nations/train.txt


100%|██████████| 1592/1592 [00:00<00:00, 287184.72it/s]


Generating Nations valid.txt dataset file in destination: datasets/data/Nations/valid.txt


100%|██████████| 199/199 [00:00<00:00, 373954.52it/s]


Generating Nations test.txt dataset file in destination: datasets/data/Nations/test.txt


...

100%|██████████| 55/55 [00:00<00:00, 772.01it/s]


Building _obs.txt files


100%|██████████| 55/55 [00:00<00:00, 770.99it/s]


Building _truth.txt files


100%|██████████| 55/55 [00:00<00:00, 5378.69it/s]


In [5]:
main()

PSL run stated at 14:43:06 on 31-Aug-2022
Building _targets.txt files


100%|██████████| 55/55 [00:00<00:00, 750.18it/s]


Adding predicates for All Predicates


100%|██████████| 55/55 [00:00<00:00, 623.54it/s]

Adding rules for All Predicates


...

Inference starting at 14:43:08 on 31-Aug-2022


4527 [pslpython.model PSL] ERROR --- (Unknown PSL logging level) -- WARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.
4539 [pslpython.model PSL] INFO --- 368  [main] INFO  org.linqs.psl.runtime.Runtime  - PSL Runtime Version 2.3.0-5547e0e
60764 [pslpython.model PSL] WARNING --- 56594 [main] WARN  org.linqs.psl.model.Model  - Rule already added to this model, skipping add: 0.73913: ( RELNGO(A, X) & (X != Y) & NGO(A, Y) ) >> NGO(X, Y) ^2
61012 [pslpython.model PSL] WARNING --- 56842 [main] WARN  org.linqs.psl.model.Model  - Rule already added to this model, skipping add: 0.839286: ( (X != Y) & NGOORGS3(A, X) & NGOORGS3(A, Y) ) >> NGO(X, Y) ^2
61090 [pslpython.model PSL] WARNING --- 56919 [main] WARN  org.linqs.psl.model.Model  - Rule already added to this model, skipping add: 0.615385: ( COMMONBLOC2(X, A) & (X != Y) & BOOKTRANSLATIONS(A, Y) ) >> RELNGO(X, Y) ^2
61187 [pslpython.model PSL] WARNING --- 57017 [main] WARN  org.linqs.psl.model.Mode

Inferenced completed at 14:52:10 on 31-Aug-2022
Writing predicates


100%|██████████| 55/55 [00:00<00:00, 2386.38it/s]

PSL run ended at 14:52:10 on 31-Aug-2022


In [3]:
from classes.Evaluation import calculate_hits_at_k
from classes.EntityConverter import EntityConverter
from classes.Dataset import get_dataset

dataset = get_dataset()
entity_converter = EntityConverter(dataset)
val_triples = dataset.validation.mapped_triples.numpy()

k =[1,3,10]
hits_at_k = calculate_hits_at_k(val_triples, entity_converter,k)

100%|██████████| 199/199 [00:00<00:00, 11972.21it/s]

hits@1	hits@3	hits@10	
0.36	0.74	0.98	
